In [130]:
import json

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F

import numpy as np
import pandas as pd

# Data import

In [131]:
with open("example-data.json", "r") as filehandle:
    data = json.load(filehandle)

# Data Parsing

In [132]:
data_parsed = [entry.replace(", ", ",") for entry in data]
data_parsed = [entry.split(",") for entry in data_parsed]
data_parsed = pd.DataFrame.from_dict(data_parsed)
data_parsed.columns = ["timestamp", "x", "temp", "humid"]
# Convert temp and humid to numeric
data_parsed["temp"] = data_parsed["temp"].astype(float)
data_parsed["humid"] = data_parsed["humid"].astype(float)
# Last row is empty
data_parsed = data_parsed[:-1]
# Remove X
data_parsed.drop(columns="x", inplace=True)
# Convert to datetime
data_parsed["timestamp"] = pd.to_datetime(data_parsed["timestamp"])


# Data Stats

In [133]:
# Time range
print(data_parsed["timestamp"].min(), data_parsed["timestamp"].max())
# Mean Temp per weekday
print(data_parsed.groupby(
    [data_parsed["timestamp"].dt.weekday])["temp"].mean())
# Mean Humid per weekday
print(data_parsed.groupby(
    [data_parsed["timestamp"].dt.weekday])["humid"].mean())

2022-11-21 11:00:00 2022-11-30 11:55:50
timestamp
0    18.461081
1    18.497826
2    18.353548
3    18.720833
4    18.228333
5    17.581667
6    16.770417
Name: temp, dtype: float64
timestamp
0    57.836216
1    56.195435
2    55.726452
3    56.201667
4    56.857500
5    58.713750
6    58.419583
Name: humid, dtype: float64


# Feature Enhancement

In [134]:
# Add hour
data_parsed["hour"] = data_parsed["timestamp"].dt.hour
# Add day of year
data_parsed["day_of_year"] = data_parsed["timestamp"].dt.day_of_year
# Add weekday
data_parsed["weekday"] = data_parsed["timestamp"].dt.weekday

# ML

In [135]:
class StartNet(nn.Module):

    def __init__(self):
        super(StartNet, self).__init__()
        self.fc1 = nn.Linear(3, 9)
        self.fc2 = nn.Linear(9, 1)            

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

startnet = StartNet()
optimizer = optim.Adam(startnet.parameters(), lr=0.001)

In [136]:
def training_loop(n_epochs, optimiser, model, loss_fn, train_dl, val_dl):
    for epoch in range(1, n_epochs + 1):
        model.train()
        for i, data in enumerate(train_dl):
            optimiser.zero_grad() # set gradients to zero
            inputs, targets = data
            output_train = model(inputs) # forwards pass
            loss_train = loss_fn(output_train, targets) # calculate loss
            loss_train.backward() # backwards pass
            optimiser.step() # update model parameters

        model.eval()
        for i, data in enumerate(val_dl):
            inputs, targets = data
            output_val = model(inputs)
            loss_val = loss_fn(output_val, targets)
        if epoch == 1 or epoch % 100 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                f" Validation loss {loss_val.item():.4f}")

In [137]:
# Train Val Split
train = data_parsed[0:int(len(data_parsed)/1.5)]
val = data_parsed[int(len(data_parsed)/1.5):]
# Full Training
train = data_parsed

class MyDataset(torch.utils.data.Dataset):

  def __init__(self, df):
 
    x = df[["hour", "weekday", "day_of_year"]].values
    y = df[["temp"]].values

    self.x_train=torch.tensor(x, dtype=torch.float32)
    self.y_train=torch.tensor(y, dtype=torch.float32)

  def __len__(self):
    return len(self.y_train)
  
  def __getitem__(self, idx):
    return self.x_train[idx], self.y_train[idx]

In [138]:
train_dl  = torch.utils.data.DataLoader(MyDataset(train), batch_size=10, shuffle=True)
val_dl    = torch.utils.data.DataLoader(MyDataset(val), batch_size=10, shuffle=True)

In [142]:
training_loop(
    n_epochs = 100, 
    optimiser = optimizer,
    model = startnet,
    loss_fn = nn.MSELoss(),
    train_dl = train_dl,
    val_dl = val_dl,
    )

Epoch 1, Training loss 0.6299, Validation loss 0.5262
Epoch 100, Training loss 0.2620, Validation loss 0.6323


In [143]:
for name, param in startnet.named_parameters():
    print(name, param)

fc1.weight Parameter containing:
tensor([[ 0.1038,  0.2970,  0.0522],
        [-0.2931,  0.2760,  0.3734],
        [-0.3823, -0.4306, -0.5033],
        [ 0.2253, -0.7034,  0.2356],
        [-0.1974,  0.1462,  0.0475],
        [ 0.4292, -0.0150,  0.2455],
        [ 0.1653,  0.1127,  0.1410],
        [-0.0168,  0.0673,  0.5731],
        [-0.1152, -0.2682,  0.1771]], requires_grad=True)
fc1.bias Parameter containing:
tensor([ 0.4414,  0.0258, -0.1899, -0.0828,  0.0698,  0.4655, -0.3765,  0.5112,
        -0.3861], requires_grad=True)
fc2.weight Parameter containing:
tensor([[ 0.0838,  0.3141,  0.1567,  0.2940, -0.1898,  0.2122, -0.2563, -0.3141,
          0.2068]], requires_grad=True)
fc2.bias Parameter containing:
tensor([0.2215], requires_grad=True)


In [144]:
startnet.eval()
for i in range(12, 23):
    for j in range(3, 6):
        for k in range(310, 320):
            print(startnet(torch.tensor([float(i), j, k])).tolist()[0])

16.963157653808594
17.01757049560547
17.071983337402344
17.126399993896484
17.18081283569336
17.235218048095703
17.289634704589844
17.34405517578125
17.39846420288086
17.452884674072266
16.731555938720703
16.785972595214844
16.84038543701172
16.894794464111328
16.949207305908203
17.003623962402344
17.05803680419922
17.112457275390625
17.16686248779297
17.221271514892578
16.499969482421875
16.554378509521484
16.608802795410156
16.663211822509766
16.717628479003906
16.77203369140625
16.82644271850586
16.880863189697266
16.93527603149414
16.989688873291016
17.013660430908203
17.068073272705078
17.12248992919922
17.176895141601562
17.231307983398438
17.285728454589844
17.34014129638672
17.394554138183594
17.44896697998047
17.503379821777344
16.782062530517578
16.836475372314453
16.89089584350586
16.9453125
16.999725341796875
17.05413055419922
17.108543395996094
17.16295623779297
17.217369079589844
17.27178955078125
16.550472259521484
16.604877471923828
16.659297943115234
16.71371078491211


In [147]:
torch.save(startnet.state_dict(), "startnet_temperature.model") 